In [5]:
import requests
from bs4 import BeautifulSoup

r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(r.text, "lxml")
#soup.find_all("table", attrs={"class": "wikitable"})

table = soup.find("table", attrs={"class": "wikitable"})

In [6]:
import pandas as pd
import json

In [7]:
rows = soup.find("table", attrs={"class": "wikitable"}).find("tbody").find_all("tr")

postcodes = []
boroughs = []
neighbourhoods = []


for row in rows:
    cells = row.find_all("td")
    if len(cells) > 0:
        postcode = cells[0].get_text()
        borough = cells[1].get_text()
        neighbourhood = cells[2].get_text().strip("\n")
        if borough != "Not assigned" and neighbourhood != "Not assigned":
            
            if postcode in postcodes:
                index = postcodes.index(postcode)
                #print(index)
                #print( postcodes[index] )
                neighbourhoods[index] += ", "+neighbourhood
            else:
                neighbourhoods.append(neighbourhood)
                postcodes.append(postcode)
                boroughs.append(borough)
#             if postcode != "M7A":
#                 toronto.append(temp)



df1 = pd.DataFrame( { 
    "PostalCode": postcodes  , 
    "Borough" : boroughs,
    "Neighbourhood":neighbourhoods
} )
df1.head(11)

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
3,North York,"Lawrence Heights, Lawrence Manor",M6A
4,Etobicoke,Islington Avenue,M9A
5,Scarborough,"Rouge, Malvern",M1B
6,North York,Don Mills North,M3B
7,East York,"Woodbine Gardens, Parkview Hill",M4B
8,Downtown Toronto,"Ryerson, Garden District",M5B
9,North York,Glencairn,M6B


In [8]:
df1.shape

(102, 3)

In [4]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]

0